Hamiltonians From Scratch*
===========================
*Its not really from scratch because I use the FermionicOp class to make it work

This is an alternative to the `qiskis_nature` Fermi-Hubbard Hamiltonian. I get the same ground state energies but a different ground state vector. To use this hamiltonian import it from the .py file [here](./defining_the_hamiltonian.py) 

In [1]:
from qiskit import *
from qiskit.quantum_info.operators import SparsePauliOp
from qiskit.quantum_info.operators import Pauli
from qiskit.quantum_info import Operator
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper

import numpy as np
from functools import *
import scipy.linalg as LA

In [ ]:
# Fermionic operators that we need
creation_ferms = [FermionicOp({f"+_{i}":1}) for i in range(0,12)]
annihilation_ferms = [FermionicOp({f"-_{i}":1}) for i in range(0,12)]
number_ferms = [creation_ferms[i] @ annihilation_ferms[i] for i in range(0, 12)]

In [ ]:

verticies = 6
spins = [0,1]
edges = [(0,1),(0,2),(0,3),(0,4),
         (5,1),(5,2),(5,3),(5,4),
         (1,2),(2,3),(3,4),(4,1)]

myTensor = lambda a, b: (a)^(b)

def swap_electron(edge:tuple[int, int], s:int, verticies:int) -> FermionicOp:
    """Swaps the electron in spin state s in vertex i and vertex j."""
    (i, j) = edge
    swap_1 = creation_ferms[i+verticies*s] @ annihilation_ferms[j+verticies*s]
    swap_2 = creation_ferms[j+verticies*s] @ annihilation_ferms[i+verticies*s]
    return swap_1 + swap_2

def number_up_down(vertex:int , verticies:int) -> FermionicOp:
    return number_ferms[vertex + verticies] @ number_ferms[vertex]

total_swap_operator:FermionicOp = reduce(lambda a, b: (a) + (b), 
                             [swap_electron(edge, s, verticies) for edge in edges for s in spins])
total_number_operator:FermionicOp = reduce(lambda a, b: (a) + (b), 
                               [number_up_down(i, verticies) for i in range(0, verticies)])

t = 1
u = 1
hamiltonian_ferm = -t*total_swap_operator + u*total_number_operator
hamiltonian = JordanWignerMapper().map(hamiltonian_ferm)


FermionicOp({'+_0 -_1': 1, '+_1 -_0': 1, '+_6 -_7': 1, '+_7 -_6': 1, '+_0 -_2': 1, '+_2 -_0': 1, '+_6 -_8': 1, '+_8 -_6': 1, '+_0 -_3': 1, '+_3 -_0': 1, '+_9 -_6': 1, '+_6 -_9': 1, '+_0 -_4': 1, '+_4 -_0': 1, '+_10 -_6': 1, '+_6 -_10': 1, '+_1 -_5': 1, '+_5 -_1': 1, '+_11 -_7': 1, '+_7 -_11': 1, '+_5 -_2': 1, '+_2 -_5': 1, '+_11 -_8': 1, '+_8 -_11': 1, '+_5 -_3': 1, '+_3 -_5': 1, '+_9 -_11': 1, '+_11 -_9': 1, '+_4 -_5': 1, '+_5 -_4': 1, '+_11 -_10': 1, '+_10 -_11': 1, '+_2 -_1': 1, '+_1 -_2': 1, '+_7 -_8': 1, '+_8 -_7': 1, '+_2 -_3': 1, '+_3 -_2': 1, '+_8 -_9': 1, '+_9 -_8': 1, '+_4 -_3': 1, '+_3 -_4': 1, '+_9 -_10': 1, '+_10 -_9': 1, '+_1 -_4': 1, '+_4 -_1': 1, '+_7 -_10': 1, '+_10 -_7': 1}, num_spin_orbitals=12, )

In [62]:
print("Dims in hamiltonian:", hamiltonian.dim)
print("hamiltonian hermitian?:", Operator(hamiltonian) == Operator(hamiltonian).adjoint())

Dims in hamiltonian: (4096, 4096)
hamiltonian hermitian?: True


In [63]:
H_matrix = hamiltonian.to_matrix()
eigs = LA.eigh(H_matrix)


In [64]:
eigenvals, eigenvectors = eigs

In [250]:
print(eigenvals[26:35])
print(eigenvals[-10:])


[-7.36710929 -7.08262565 -7.05051053 -7.05051053 -6.91125277 -6.91125277
 -6.91125277 -6.91125277 -6.91125277]
[11.04524552 11.04524552 11.07417788 11.36486336 11.36486336 11.36486336
 11.36486336 11.36486336 11.36486336 12.18284163]


In [260]:
ground_state = [(x, f"{i:012b}") for i,x in enumerate(eigenvectors[:, 8]) if abs(x) > 1e-8]
(ground_state)

[(np.complex128(-0.07923727778936213-0j), '000011000011'),
 (np.complex128(-0.07957568456834961-0j), '000011000101'),
 (np.complex128(0.012219776107149303-0j), '000011000110'),
 (np.complex128(0.09637489189435634-0j), '000011001010'),
 (np.complex128(0.09332685866424223-0j), '000011001100'),
 (np.complex128(-0.013219787441995997-0j), '000011010001'),
 (np.complex128(0.08057569590320274-0j), '000011010010'),
 (np.complex128(0.07673866282010688-0j), '000011010100'),
 (np.complex128(-0.015343701226979716-0j), '000011011000'),
 (np.complex128(-0.09637489189434974-0j), '000011100001'),
 (np.complex128(-0.0157838319854981-0j), '000011100100'),
 (np.complex128(-0.10863962566456449-0j), '000011101000'),
 (np.complex128(-0.09288672790571646-0j), '000011110000'),
 (np.complex128(-0.07957568456836259-0j), '000101000011'),
 (np.complex128(-0.043285490932380155-0j), '000101000101'),
 (np.complex128(0.031057937524333747-0j), '000101000110'),
 (np.complex128(0.028883665933549522-0j), '000101001001'),

In [ ]:
(c0, _) = ground_state[0] # same node
c0_elements = [(c, i) for c,i in ground_state if c0-1e-5 < c < c0+1e-5]

(c1, _) = ground_state[1] # adjacemt nodes
c1_elements = [(c, i) for c,i in ground_state if c1-1e-5 < c < c1+1e-5]

(c2, _) = ground_state[19] # oposite nodes
c2_elements = [(c, i) for c,i in ground_state if c2-1e-5 < c < c2+1e-5]

print(f"c0:{c0}")
print(f"c1:{c1}")
print(f"c2:{c2}")

ground_state[1]

c0:(-0.15253406813364298+0j)
c1:(-0.16869036372022123+0j)
c2:(-0.17197171126969088+0j)


(np.complex128(-0.16869036372022123+0j), '000001000010')

In [210]:
def num_of_ones(s:str) -> int:
    return len(list(filter(lambda x: x=='1', s)))

len(list(filter(lambda x: x==4, [num_of_ones(s) for c,s in ground_state])))

438